In [2]:
# INITIAL PACKAGES
import os
import numpy as np
import pandas as pd

from utils import load_data, run

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [4]:
gdrive_root = '/content/gdrive/My Drive'
data_path = os.path.join(gdrive_root, 'final_project')
os.listdir(data_path)

['sample_submission.csv',
 'test_id.csv',
 'test_images.npy',
 'train_id_label.csv',
 'train_images.npy',
 'valid_images.npy',
 'valid_id_label.csv']

In [5]:
train_data, valid_data, test_data = load_data(data_path)

In [6]:
print(f'train id label:\n {train_data[0].head()}')
print(f'train images shape: {train_data[1].shape}\n')
assert len(train_data[0]) == len(train_data[1])

print(f'valid id label:\n {valid_data[0].head()}')
print(f'valid images shape: {valid_data[1].shape}\n')
assert len(valid_data[0]) == len(valid_data[1])

print(f'test id:\n {test_data[0].head()}')
print(f'test images shape: {test_data[1].shape}\n')
assert len(test_data[0]) == len(test_data[1])

train id label:
            id  label
0  a1cec2874d      1
1  ddbe361041      7
2  910628fd4e      2
3  171ae22c4b     10
4  f1e68a9c42      1
train images shape: (100000, 3, 32, 32)

valid id label:
            id  label
0  f0829f4147      7
1  91116a7846     11
2  88c83f1240      3
3  a7cd83fe4f      4
4  78b3ce0c46      1
valid images shape: (10000, 3, 32, 32)

test id:
            id
0  edf4ea9a4b
1  7496e3e847
2  e0f2110942
3  9fb87df04a
4  0a3608ca47
test images shape: (10000, 3, 32, 32)



In [14]:
# IMPORT PACKAGES YOU NEED
from time import time
import torch
from torch.utils.data import TensorDataset, DataLoader
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import init
import math

import torch.nn.parallel
import torch.optim
import torch.utils.data
from tqdm import tqdm



class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion*planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=14):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

def ResNet18():
    return ResNet(BasicBlock, [2,2,2,2])

def ResNet34():
    return ResNet(BasicBlock, [3,4,6,3])

def ResNet50():
    return ResNet(Bottleneck, [3,4,6,3])

def ResNet101():
    return ResNet(Bottleneck, [3,4,23,3])

def ResNet152():
    return ResNet(Bottleneck, [3,8,36,3])
def DMI_loss(output, target):
    outputs = F.softmax(output, dim=1)
    targets = target.reshape(target.size(0), 1).cpu()
    y_onehot = torch.FloatTensor(target.size(0), 14).zero_()
    y_onehot.scatter_(1, targets, 1)
    y_onehot = y_onehot.transpose(0, 1).cuda()
    mat = y_onehot @ outputs
    mat = mat / target.size(0)
    return -1.0 * torch.log(torch.abs(torch.det(mat.float())) + 0.001)     

In [7]:
torch.cuda.empty_cache()

In [13]:
def train_and_predict(train_data, valid_data, test_data):
    """Train a model and return prediction on test images.

    Given train and valid data, build your model and optimize.
    Then, return predictions on test_images.

    You can import packages you want inside 'EDIT HERE' as long as they are permitted.
    (See document for the list of possible packages)

    arguments:
        train_data: tuple of (pandas.DataFrame, np.array).
        - 0: pandas.DataFrame with columns ['id', 'label']
          'id' contains unique id assigned to each image.
          'label' contains label (0 ~ # classes-1) corresponding to its image.
        - 1: train image in np.array of (# train data, # channel, height, width)

        valid_data: tuple of (pandas.DataFrame, np.array).
        - 0: pandas.DataFrame with columns ['id', 'label']
          'id' contains unique id assigned to each image.
          'label' contains label (0 ~ # classes-1) corresponding to its image.
        - 1: valid image in np.array of (# valid data, # channel, height, width)

        test_data: tuple of (pandas.DataFrame, np.array).
        - 0: pandas.DataFrame with columns ['id']
          'id' contains unique id assigned to each image.
        - 1: test image in np.array of (# test data, # channel, height, width)
    
    returns:
        pandas.DataFrame, predictions on test images with columns ['id', 'label'].
        'id' should contain unique id assigned to test images. 
        'label' should contain prediction on the test image correspond to its id

    """

    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    LEARNING_RATE = 0.00001
    BATCH_SIZE = 128
    TEST_BATCH_SIZE = 1024
    NUM_EPOCHS = 150
    PATIENCE = 10
    ENDURE = 0

    train_id_label, train_images = train_data
    valid_id_label, valid_images = valid_data
    test_id, test_images = test_data

    num_train = len(train_images)
    num_valid = len(valid_images)
    num_test = len(test_images)
    
    # Convert data into torch.Tensor
    x_train = torch.FloatTensor(train_images)
    y_train = train_id_label['label'].values
    y_train = torch.LongTensor(y_train)

    x_valid = torch.FloatTensor(valid_images)
    y_valid = valid_id_label['label'].values
    y_valid = torch.LongTensor(y_valid)

    x_test = torch.FloatTensor(test_images)

    num_features = np.prod(x_train.shape[1:])
    num_classes = int(y_train.max()) + 1

    # Build torch dataset, dataloader
    train_dataset = TensorDataset(x_train, y_train)
    valid_dataset = TensorDataset(x_valid, y_valid)
    test_dataset = TensorDataset(x_test)

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    valid_loader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=TEST_BATCH_SIZE, shuffle=False)

    # Build CNN model
    model = ResNet50().cuda()
    #odel.load_state_dict(torch.load('best_mlp_real.p'))


    # Optimizer and loss function
    optimizer = torch.optim.SGD(model.parameters(), momentum=0.9, weight_decay=1e-4, lr=LEARNING_RATE)
    
    loss_fn = torch.nn.CrossEntropyLoss()

    # Train model
    mean_train_losses = []
    mean_valid_losses = []
    valid_acc_list = []
    best_acc = -1

    train_s = time()
    for epoch in range(1, NUM_EPOCHS + 1):
        epoch_s = time()
        model.train()
        
        train_losses = []
        valid_losses = []



        for i, (images, labels) in enumerate(train_loader):
            #images, labels = images.to(device), labels.to(device)

            images = torch.autograd.Variable(images.cuda())
            labels = torch.autograd.Variable(labels.cuda())
          
            optimizer.zero_grad()
            
            outputs = model(images)

            loss = loss_fn(outputs,labels)
            loss.backward()
            optimizer.step()
            
            train_losses.append(loss.item())
            
        model.eval()
        
        correct = 0
        total = 0
        # Disable gradient calculation for memory, computation efficiency
        with torch.no_grad():
            for i, (images, labels) in enumerate(valid_loader):
                #images, labels = images.to(device), labels.to(device)
                images = torch.autograd.Variable(images.cuda())
                labels = torch.autograd.Variable(labels.cuda())
            
                outputs = model(images)
                loss = loss_fn(outputs, labels)
                
                valid_losses.append(loss.item())
                
                predicted = torch.argmax(outputs.data, 1)
                correct += (predicted == labels).sum().item()
                total += labels.size(0)
                
        mean_train_losses.append(np.mean(train_losses))
        mean_valid_losses.append(np.mean(valid_losses))

        epoch_elapsed = time() - epoch_s
        
        valid_acc = correct / total
        valid_acc_list.append(valid_acc)
        print('epoch: {}, train loss: {:.4f}, valid loss: {:.4f}, valid acc: {:.4f}, elapsed: {:.4f}'\
            .format(epoch, np.mean(train_losses), np.mean(valid_losses), valid_acc, epoch_elapsed))
        
        if best_acc < valid_acc:
            print('Best Accuracy updated (%.4f => %.4f)' % (best_acc, valid_acc))
            best_acc = valid_acc
            best_epoch = epoch
            ENDURE = 0
            # Save best model
            torch.save(model.state_dict(), 'best_mlp.p')
        else:
            ENDURE += 1
            if ENDURE >= PATIENCE:
                print('Early stop triggered...!')
                break
    train_elapsed = time() - train_s
    print('Training Finished...!!')
    print('Time: %.4f' % train_elapsed)
    print('Best Valid acc : %.4f at epoch %d' % (best_acc, best_epoch))
    
    # Load best model
    model.load_state_dict(torch.load('best_mlp.p'))
    model.eval()

    # Make prediction on test data
    test_preds = []
    with torch.no_grad():
        for i, (images, ) in enumerate(test_loader):
            #images = images.to(device)
            images = torch.autograd.Variable(images.cuda())
        
            outputs = model(images)
            
            pred = outputs.argmax(1)

            if device == 'cuda':
                test_preds += pred.detach().cpu().numpy().tolist()
            else:
                test_preds += pred.detach().numpy().tolist()
    
    # Return prediction
    test_id['label'] = test_preds
    pred = test_id.loc[:, ['id', 'label']]

    return pred

In [ ]:
run(train_and_predict, train_data, valid_data, test_data)

In [14]:
def train_and_predict_again(train_data, valid_data, test_data):
    """Train a model and return prediction on test images.

    Given train and valid data, build your model and optimize.
    Then, return predictions on test_images.

    You can import packages you want inside 'EDIT HERE' as long as they are permitted.
    (See document for the list of possible packages)

    arguments:
        train_data: tuple of (pandas.DataFrame, np.array).
        - 0: pandas.DataFrame with columns ['id', 'label']
          'id' contains unique id assigned to each image.
          'label' contains label (0 ~ # classes-1) corresponding to its image.
        - 1: train image in np.array of (# train data, # channel, height, width)

        valid_data: tuple of (pandas.DataFrame, np.array).
        - 0: pandas.DataFrame with columns ['id', 'label']
          'id' contains unique id assigned to each image.
          'label' contains label (0 ~ # classes-1) corresponding to its image.
        - 1: valid image in np.array of (# valid data, # channel, height, width)

        test_data: tuple of (pandas.DataFrame, np.array).
        - 0: pandas.DataFrame with columns ['id']
          'id' contains unique id assigned to each image.
        - 1: test image in np.array of (# test data, # channel, height, width)
    
    returns:
        pandas.DataFrame, predictions on test images with columns ['id', 'label'].
        'id' should contain unique id assigned to test images. 
        'label' should contain prediction on the test image correspond to its id

    """

    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    LEARNING_RATE = 0.0001
    BATCH_SIZE = 64
    TEST_BATCH_SIZE = 1024
    NUM_EPOCHS = 150
    PATIENCE = 10
    ENDURE = 0

    train_id_label, train_images = train_data
    valid_id_label, valid_images = valid_data
    test_id, test_images = test_data

    num_train = len(train_images)
    num_valid = len(valid_images)
    num_test = len(test_images)
    
    # Convert data into torch.Tensor
    x_train = torch.FloatTensor(train_images)
    y_train = train_id_label['label'].values
    y_train = torch.LongTensor(y_train)

    x_valid = torch.FloatTensor(valid_images)
    y_valid = valid_id_label['label'].values
    y_valid = torch.LongTensor(y_valid)

    x_test = torch.FloatTensor(test_images)

    num_features = np.prod(x_train.shape[1:])
    num_classes = int(y_train.max()) + 1

    # Build torch dataset, dataloader
    train_dataset = TensorDataset(x_train, y_train)
    valid_dataset = TensorDataset(x_valid, y_valid)
    test_dataset = TensorDataset(x_test)

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    valid_loader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=TEST_BATCH_SIZE, shuffle=False)

    # Build CNN model
    model = ResNet50().cuda()
    model.load_state_dict(torch.load('best_mlp.p'))


    # Optimizer and loss function
    optimizer = torch.optim.SGD(model.parameters(), momentum=0.9, weight_decay=1e-4, lr=LEARNING_RATE)
    
    loss_fn = DMI_loss

    # Train model
    mean_train_losses = []
    mean_valid_losses = []
    valid_acc_list = []
    best_acc = -1

    train_s = time()
    for epoch in range(1, NUM_EPOCHS + 1):
        epoch_s = time()
        model.train()

        if(epoch >= 5) :
          LEARNING_RATE = 0.00005
          optimizer.lr = 0.00005
        
        train_losses = []
        valid_losses = []



        for i, (images, labels) in enumerate(train_loader):
            #images, labels = images.to(device), labels.to(device)

            images = torch.autograd.Variable(images.cuda())
            labels = torch.autograd.Variable(labels.cuda())
          
            optimizer.zero_grad()
            
            outputs = model(images)

            loss = loss_fn(outputs,labels)
            loss.backward()
            optimizer.step()
            
            train_losses.append(loss.item())
            
        model.eval()
        
        correct = 0
        total = 0
        # Disable gradient calculation for memory, computation efficiency
        with torch.no_grad():
            for i, (images, labels) in enumerate(valid_loader):
                #images, labels = images.to(device), labels.to(device)
                images = torch.autograd.Variable(images.cuda())
                labels = torch.autograd.Variable(labels.cuda())
            
                outputs = model(images)
                loss = loss_fn(outputs, labels)
                
                valid_losses.append(loss.item())
                
                predicted = torch.argmax(outputs.data, 1)
                correct += (predicted == labels).sum().item()
                total += labels.size(0)
                
        mean_train_losses.append(np.mean(train_losses))
        mean_valid_losses.append(np.mean(valid_losses))

        epoch_elapsed = time() - epoch_s
        
        valid_acc = correct / total
        valid_acc_list.append(valid_acc)
        print('epoch: {}, train loss: {:.4f}, valid loss: {:.4f}, valid acc: {:.4f}, elapsed: {:.4f}'\
            .format(epoch, np.mean(train_losses), np.mean(valid_losses), valid_acc, epoch_elapsed))
        
        if best_acc < valid_acc:
            print('Best Accuracy updated (%.4f => %.4f)' % (best_acc, valid_acc))
            best_acc = valid_acc
            best_epoch = epoch
            ENDURE = 0
            # Save best model
            torch.save(model.state_dict(), 'best_mlp_real.p')
        else:
            ENDURE += 1
            if ENDURE >= PATIENCE:
                print('Early stop triggered...!')
                break
    train_elapsed = time() - train_s
    print('Training Finished...!!')
    print('Time: %.4f' % train_elapsed)
    print('Best Valid acc : %.4f at epoch %d' % (best_acc, best_epoch))
    
    # Load best model
    model.load_state_dict(torch.load('best_mlp_real.p'))
    model.eval()

    # Make prediction on test data
    test_preds = []
    with torch.no_grad():
        for i, (images, ) in enumerate(test_loader):
            #images = images.to(device)
            images = torch.autograd.Variable(images.cuda())
        
            outputs = model(images)
            
            pred = outputs.argmax(1)

            if device == 'cuda':
                test_preds += pred.detach().cpu().numpy().tolist()
            else:
                test_preds += pred.detach().numpy().tolist()
    
    # Return prediction
    test_id['label'] = test_preds
    pred = test_id.loc[:, ['id', 'label']]

    return pred

In [15]:
run(train_and_predict_again, train_data, valid_data, test_data)

epoch: 1, train loss: 0.4247, valid loss: 2.0544, valid acc: 0.4968, elapsed: 720.5079
Best Accuracy updated (-1.0000 => 0.4968)


KeyboardInterrupt: ignored